## Live Session Module 7

### H. Diana McSpadden (hdm5s)

In [1]:
import numpy as np
import pandas as pd
import wget
import sqlite3
import sqlalchemy
import requests
import json
import os
import sys
import dotenv


sys.tracebacklimit = 0 # turn off the error tracebacks

In [10]:
nba_db= sqlite3.connect('nba.db')

In [11]:
# read all the files in the database
myquery = """
SELECT * FROM games"""

pd.read_sql_query(myquery, nba_db)

,game_id,game_date,ot,season
0,202202170BRK,2022-02-17,0,2022
1,202202170CHO,2022-02-17,2,2022
2,202202170LAC,2022-02-17,0,2022
3,202202170MIL,2022-02-17,0,2022
4,202202170NOP,2022-02-17,0,2022
...,...,...,...,...
3192,202001080GSW,2020-01-08,0,2020
3193,202008020HOU,2020-08-02,0,2020
3194,201911060HOU,2019-11-06,0,2020
3195,201912250GSW,2019-12-25,0,2020


In [15]:
# player's individual stats for all games in the 2022 season

myquery = """
SELECT * 
FROM player_game pg
INNER JOIN players p
    ON pg.player_id = p.player_id
INNER JOIN games g
    ON pg.game_id = g.game_id
WHERE g.season = 2022
"""

pd.read_sql_query(myquery, nba_db)

,game_id,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,...,sf%,pf%,c%,active_position_minutes,player_id,player,game_id,game_date,ot,season
0,202202170BRK,aldrila01,0,28:19,5,8,0.625,0,0,0.000,...,0.0,0.0,100.0,70.877559,aldrila01,LaMarcus Aldridge,202202170BRK,2022-02-17,0,2022
1,202202170BRK,avdijde01,0,30:50,4,10,0.400,1,3,0.333,...,62.0,31.0,0.0,54.944529,avdijde01,Deni Avdija,202202170BRK,2022-02-17,0,2022
2,202202170BRK,brownbr01,1,34:57,3,10,0.300,0,1,0.000,...,58.0,11.0,0.0,43.397040,brownbr01,Bruce Brown,202202170BRK,2022-02-17,0,2022
3,202202170BRK,bryanth01,1,14:04,5,6,0.833,0,1,0.000,...,0.0,0.0,100.0,36.472537,bryanth01,Thomas Bryant,202202170BRK,2022-02-17,0,2022
4,202202170BRK,caldwke01,1,25:26,3,7,0.429,1,3,0.333,...,67.0,0.0,0.0,47.021807,caldwke01,Kentavious Caldwell-Pope,202202170BRK,2022-02-17,0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23856,202201210PHI,niangge01,0,30:13,3,11,0.273,1,7,0.143,...,33.0,64.0,3.0,44.840107,niangge01,Georges Niang,202201210PHI,2022-01-21,0,2022
23857,202201210PHI,powelmy01,0,0:00,0,0,0.000,0,0,0.000,...,0.0,0.0,0.0,48.463595,powelmy01,Myles Powell,202201210PHI,2022-01-21,0,2022
23858,202201210PHI,sprinja01,0,0:00,0,0,0.000,0,0,0.000,...,63.0,0.0,0.0,36.448977,sprinja01,Jaden Springer,202201210PHI,2022-01-21,0,2022
23859,202201210PHI,winslju01,0,0:00,0,0,0.000,0,0,0.000,...,8.0,72.0,20.0,62.753626,winslju01,Justise Winslow,202201210PHI,2022-01-21,0,2022


In [57]:
# who should have been MVP in 2022?
myquery = """
SELECT p.player, SUM(pg.fg) AS total_field_goals, SUM(pg.fga) AS total_field_goal_attempts, ((SUM(pg.fg) * 1.0) / (SUM(pg.fga) * 1.0)) AS field_goal_percentage
FROM player_game pg
INNER JOIN players p
    ON pg.player_id = p.player_id
INNER JOIN games g
    ON pg.game_id = g.game_id
WHERE g.season = 2022
GROUP BY p.player_id, p.player
HAVING total_field_goal_attempts > 100
ORDER BY field_goal_percentage DESC, total_field_goals DESC
LIMIT 5
"""

pd.read_sql_query(myquery, nba_db)


,player,total_field_goals,total_field_goal_attempts,field_goal_percentage
0,Mitchell Robinson,189,248,0.762097
1,Robert Williams,208,284,0.732394
2,Onyeka Okongwu,92,128,0.718750
3,Rudy Gobert,246,346,0.710983
4,Daniel Gafford,183,272,0.672794


In [58]:
# new system for the NBA scoring system from fantasy basketball
#
#Point = 1
#3PM = 1
#FGA = -1
#FGM = 2
#FTA = -1
#FTM = 1
#REB = 1
#AST = 2
#STL = 4
#BLK = 4
#TOV = -2

myquery = """
SELECT 
    p.player, 
    1.0*SUM(pg.pts) AS pts, 
    1.0*SUM(pg.fg3) AS fg3, 
    1.0*SUM(pg.fg) AS fg,
    1.0*SUM(pg.fga) AS fga,
    1.0*SUM(pg.fta) AS fta,
    1.0*SUM(pg.ft) AS ft,
    1.0*SUM(pg.trb) AS trb,
    1.0*SUM(pg.ast) AS ast,
    1.0*SUM(pg.stl) AS stl,
    1.0*SUM(pg.blk) AS blk,
    1.0*SUM(pg.tov) AS tov,
    SUM(pg.pts) + SUM(pg.fg3) + 2*SUM(pg.fg) - SUM(pg.fga) - 
    SUM(pg.fta)  + SUM(pg.ft)  + SUM(pg.trb) + 2*SUM(pg.ast) + 
    4*SUM(pg.stl) + 4*SUM(pg.blk) - 2*SUM(pg.tov) AS mvpscore
FROM player_game pg
INNER JOIN players p
    ON pg.player_id = p.player_id
INNER JOIN games g
    ON pg.game_id = g.game_id
WHERE g.season = 2022
GROUP BY p.player_id, p.player
ORDER BY mvpscore DESC
LIMIT 5
"""

pd.read_sql_query(myquery, nba_db)


,player,pts,fg3,fg,fga,fta,ft,trb,ast,stl,blk,tov,mvpscore
0,Nikola Jokic,1352.0,84.0,516.0,906.0,292.0,236.0,718.0,410.0,69.0,39.0,197.0,3082
1,Giannis Antetokounmpo,1443.0,58.0,496.0,906.0,545.0,393.0,551.0,296.0,48.0,68.0,162.0,2718
2,Dejounte Murray,1036.0,67.0,428.0,932.0,149.0,113.0,432.0,486.0,106.0,17.0,125.0,2637
3,Chris Paul,866.0,60.0,325.0,667.0,185.0,156.0,259.0,619.0,108.0,18.0,138.0,2605
4,Trae Young,1475.0,158.0,495.0,1085.0,364.0,327.0,206.0,493.0,53.0,5.0,217.0,2491


In [ ]:
# try to get the win/loss record for 2022 season
